In [5]:
!pip install rouge-score
!pip install accelerate -U
!pip install gradio

In [7]:
import pandas as pd
import re
import os

In [9]:
file_path = "/Users/delvefohoue/Downloads/OneDrive_Bia_1_6-15-2024/IndianFoodDataset.csv"
if os.path.exists(file_path):
    data = pd.read_csv(file_path)
    print(data.head())
else:
    print(f"File not found: {file_path}")

   Srno                                         RecipeName  \
0     1                               Masala Karela Recipe   
1     2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2     3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3     4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4     5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   

                                TranslatedRecipeName  \
0                               Masala Karela Recipe   
1                         Spicy Tomato Rice (Recipe)   
2  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3  Gongura Chicken Curry Recipe - Andhra Style Go...   
4  Andhra Style Alam Pachadi Recipe - Adrak Chutn...   

                                         Ingredients  \
0  6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...   
1  2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...   
2  1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...   
3  500 grams Chicken,2 Onion - chopped,1 Tomato -...   
4  1 बड़ा च

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186 entries, 0 to 1185
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Srno                    1186 non-null   int64 
 1   RecipeName              1186 non-null   object
 2   TranslatedRecipeName    1186 non-null   object
 3   Ingredients             1185 non-null   object
 4   TranslatedIngredients   1185 non-null   object
 5   PrepTimeInMins          1186 non-null   int64 
 6   CookTimeInMins          1186 non-null   int64 
 7   TotalTimeInMins         1186 non-null   int64 
 8   Servings                1186 non-null   int64 
 9   Cuisine                 1186 non-null   object
 10  Course                  1186 non-null   object
 11  Diet                    1186 non-null   object
 12  Instructions            1186 non-null   object
 13  TranslatedInstructions  1186 non-null   object
 14  URL                     1186 non-null   object
dtypes: i

In [13]:
# Drop specified columns
data = data.drop(['Srno', 'RecipeName', 'Ingredients', 'Instructions'], axis=1)

In [15]:
## Viewing the dataframe after dropping unwanted columns
data.head()

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL
0,Masala Karela Recipe,"6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...",15,30,45,6,Indian,Side Dish,Diabetic Friendly,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...
1,Spicy Tomato Rice (Recipe),"2-1 / 2 cups rice - cooked, 3 tomatoes, 3 teas...",5,10,15,3,South Indian Recipes,Main Course,Vegetarian,"To make tomato puliogere, first cut the tomato...",http://www.archanaskitchen.com/spicy-tomato-ri...
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...",20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,"To begin making the Ragi Vermicelli Recipe, fi...",http://www.archanaskitchen.com/ragi-vermicelli...
3,Gongura Chicken Curry Recipe - Andhra Style Go...,"500 grams Chicken,2 Onion - chopped,1 Tomato -...",15,30,45,4,Andhra,Lunch,Non Vegeterian,To begin making Gongura Chicken Curry Recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,Andhra Style Alam Pachadi Recipe - Adrak Chutn...,"1 tablespoon chana dal, 1 tablespoon white ura...",10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,"To make Andhra Style Alam Pachadi, first heat ...",https://www.archanaskitchen.com/andhra-style-a...


In [17]:
print(data.loc[0,'TranslatedInstructions'])

To begin making the Masala Karela Recipe,de-seed the karela and slice. Do not remove the skin as the skin has all the nutrients. Add the karela to the pressure cooker with 3 tablespoon of water, salt and turmeric powder and pressure cook for three whistles. Release the pressure immediately and open the lids. Keep aside.Heat oil in a heavy bottomed pan or a kadhai. Add cumin seeds and let it sizzle.Once the cumin seeds have sizzled, add onions and saute them till it turns golden brown in color.Add the karela, red chilli powder, amchur powder, coriander powder and besan. Stir to combine the masalas into the karela.Drizzle a little extra oil on the top and mix again. Cover the pan and simmer Masala Karela stirring occasionally until everything comes together well. Turn off the heat.Transfer Masala Karela into a serving bowl and serve.Serve Masala Karela along with Panchmel Dal and Phulka for a weekday meal with your family. 


In [19]:
## Checking if there is any missing values in the dataframe
data.isnull().sum()

TranslatedRecipeName      0
TranslatedIngredients     1
PrepTimeInMins            0
CookTimeInMins            0
TotalTimeInMins           0
Servings                  0
Cuisine                   0
Course                    0
Diet                      0
TranslatedInstructions    0
URL                       0
dtype: int64

In [21]:
## Extracting the missing value data.
miss_val = data[data.TranslatedIngredients.isnull()]

In [23]:
## Displaying the missing value records.
miss_val

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL
287,Pear And Walnut Salad Recipe,NaN,10,30,40,2,Continental,Appetizer,Vegetarian,To begin making the Pear And Walnut Salad Reci...,https://www.archanaskitchen.com/pear-and-walnu...


In [25]:
# Drop rows with missing values
data = data.dropna()

In [27]:
print(data.shape)

(1185, 11)


In [29]:
def clean_text(text):
    # Remove non-alphanumeric characters and extra whitespaces
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip().lower()
    return text

# Apply text cleaning to relevant columns
text_columns = ['TranslatedRecipeName', 'TranslatedIngredients', 'TranslatedInstructions']
for column in text_columns:
    data[column] = data[column].apply(clean_text)

In [31]:
## Displaying some sample rows after cleaning the data.
data.head()

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL
0,masala karela recipe,6 karela bitter gourd pavakkai deseededsalt to...,15,30,45,6,Indian,Side Dish,Diabetic Friendly,to begin making the masala karela recipedeseed...,https://www.archanaskitchen.com/masala-karela-...
1,spicy tomato rice recipe,21 2 cups rice cooked 3 tomatoes 3 teaspoons b...,5,10,15,3,South Indian Recipes,Main Course,Vegetarian,to make tomato puliogere first cut the tomatoe...,http://www.archanaskitchen.com/spicy-tomato-ri...
2,ragi semiya upma recipe ragi millet vermicelli...,112 cups rice vermicelli noodles thin1 onion s...,20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,to begin making the ragi vermicelli recipe fir...,http://www.archanaskitchen.com/ragi-vermicelli...
3,gongura chicken curry recipe andhra style gong...,500 grams chicken2 onion chopped1 tomato chopp...,15,30,45,4,Andhra,Lunch,Non Vegeterian,to begin making gongura chicken curry recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,andhra style alam pachadi recipe adrak chutney...,1 tablespoon chana dal 1 tablespoon white urad...,10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...


In [33]:
data.loc[0,'TranslatedInstructions']

'to begin making the masala karela recipedeseed the karela and slice do not remove the skin as the skin has all the nutrients add the karela to the pressure cooker with 3 tablespoon of water salt and turmeric powder and pressure cook for three whistles release the pressure immediately and open the lids keep asideheat oil in a heavy bottomed pan or a kadhai add cumin seeds and let it sizzleonce the cumin seeds have sizzled add onions and saute them till it turns golden brown in coloradd the karela red chilli powder amchur powder coriander powder and besan stir to combine the masalas into the kareladrizzle a little extra oil on the top and mix again cover the pan and simmer masala karela stirring occasionally until everything comes together well turn off the heattransfer masala karela into a serving bowl and serveserve masala karela along with panchmel dal and phulka for a weekday meal with your family'

In [35]:
# Save the cleaned dataset
data.to_csv("/Users/delvefohoue/cleanedk_IndianFoodDataset.csv", index=False)

In [37]:
os.getcwd()

'/Users/delvefohoue/Downloads'

In [39]:
os.chdir("/Users/delvefohoue/Downloads")

In [41]:
data.shape

(1185, 11)

In [43]:
#Training GPT2 Model on cleaned_IndianFoodDataset


In [45]:
pip install --upgrade pyarrow


Note: you may need to restart the kernel to use updated packages.


In [47]:
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


In [49]:
# Load the cleaned dataset

data = pd.read_csv("/Users/delvefohoue/cleanedk_IndianFoodDataset.csv")


In [59]:
data.head()

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL
0,masala karela recipe,6 karela bitter gourd pavakkai deseededsalt to...,15,30,45,6,Indian,Side Dish,Diabetic Friendly,to begin making the masala karela recipedeseed...,https://www.archanaskitchen.com/masala-karela-...
1,spicy tomato rice recipe,21 2 cups rice cooked 3 tomatoes 3 teaspoons b...,5,10,15,3,South Indian Recipes,Main Course,Vegetarian,to make tomato puliogere first cut the tomatoe...,http://www.archanaskitchen.com/spicy-tomato-ri...
2,ragi semiya upma recipe ragi millet vermicelli...,112 cups rice vermicelli noodles thin1 onion s...,20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,to begin making the ragi vermicelli recipe fir...,http://www.archanaskitchen.com/ragi-vermicelli...
3,gongura chicken curry recipe andhra style gong...,500 grams chicken2 onion chopped1 tomato chopp...,15,30,45,4,Andhra,Lunch,Non Vegeterian,to begin making gongura chicken curry recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,andhra style alam pachadi recipe adrak chutney...,1 tablespoon chana dal 1 tablespoon white urad...,10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...


In [61]:
# Split the dataset into train, validation, and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, validation_data = train_test_split(train_data, test_size=0.1, random_state=42)



In [63]:
# Save the split datasets

train_data.to_csv("train_dataset.csv", index=False)
validation_data.to_csv("validation_dataset.csv", index=False)
test_data.to_csv("test_dataset.csv", index=False)

In [70]:
# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [72]:
# Tokenize the datasets
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="train_dataset.csv",
    block_size=128
)

validation_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="validation_dataset.csv",
    block_size=128
)

test_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="test_dataset.csv",
    block_size=128
)

/opt/anaconda3/lib/python3.11/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [74]:
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-indian-food",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

In [76]:
# Set up trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    ),
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

In [78]:
# Start training

trainer.train()

Step,Training Loss
500,3.097400
1000,2.702500
1500,2.475600
2000,2.352600
2500,2.318600
3000,2.189200
3500,2.193800


TrainOutput(global_step=3702, training_loss=2.4570145650788424, metrics={'train_runtime': 11650.0087, 'train_samples_per_second': 0.636, 'train_steps_per_second': 0.318, 'total_flos': 483651551232000.0, 'train_loss': 2.4570145650788424, 'epoch': 3.0})

In [80]:
# Save the model

trainer.save_model("gpt2-indian-food")

# Save the configuration file
model = trainer.model
model.config.save_pretrained("gpt2-indian-food_config.json")

In [82]:
# Evaluate on the test set

results = trainer.evaluate(test_dataset)
print(results)

{'eval_loss': 2.3359367847442627, 'eval_runtime': 142.209, 'eval_samples_per_second': 4.803, 'eval_steps_per_second': 0.605, 'epoch': 3.0}


In [84]:
## MODEL EVALUATION


In [86]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/delvefohoue/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/delvefohoue/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/delvefohoue/nltk_data...


True

In [94]:
# Load the trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2-indian-food")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Function to generate text using the model
def generate_text(prompt, max_length=100, temperature=1.0):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=max_length, temperature=temperature, num_beams=5, no_repeat_ngram_size=2)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

# Example prompts and references
prompts = ["To make Masala Karela,",
           "Tomato Puliogere is prepared by",
           "Ragi Vermicelli Recipe starts with",
           "Gongura Chicken Curry Recipe first prep all the ingredients",
           "Andhra Style Alam Pachadi is made by"]

references = ['karela_text',
              "To make tomato puliogere, first cut the tomatoes.",
              "To begin making the Ragi Vermicelli Recipe, first steam the ragi vermicelli.",
              "To begin making Gongura Chicken Curry Recipe, first prep all the ingredients.",
              "To make Andhra Style Alam Pachadi, first heat oil in a pan."]

# Generate text and calculate BLEU and ROUGE scores
smooth = SmoothingFunction().method1  # Define smoothing function for BLEU
bleu_scores = []
rouge_scores = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for prompt, reference in zip(prompts, references):
    generated_text = generate_text(prompt, 500, 0.5)

    # BLEU score
    bleu_score = sentence_bleu([reference.split()], generated_text.split(), smoothing_function=smooth)
    bleu_scores.append(bleu_score)
    print(f"Prompt: {prompt}\nGenerated: {generated_text}\nReference: {reference}\nBLEU Score: {bleu_score:.4f}\n")

    # ROUGE scores
    rouge_score = rouge_scores.score(generated_text, reference)
    print(f"ROUGE Scores: {rouge_score}\n")

# Average BLEU score
average_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {average_bleu:.4f}")

/opt/anaconda3/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: To make Masala Karela,
Generated: To make Masala Karela, heat oil in a heavy bottomed pan add mustard seeds cumin seeds and let it splutteradd onions and saute till they turn translucentadd tomatoes and cook till the tomatoes become soft and mushyadd turmeric powder red chilli powder coriander powder garam masala powder and salt to taste mix well and allow it to cook for 5 to 6 minutes until the raw smell of the spices goes awayadd the cooked masalas and give it a good stir and turn off the heat allow the pressure to release naturally open the lid and keep it asidein a large mixing bowl combine all the ingredients mentioned in the list and combine them well to form a thick batterheat a kadai with oil add the mustard oil and fry till it turns golden brown and crisp on both sidesonce the oil is hot add ginger garlic paste and stir fry for a few seconds until it becomes fragrantadd chopped onions green chillies and green chilies and mix everything welladd tomato puree and toss wel

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Tomato Puliogere is prepared by
Generated: Tomato Puliogere is prepared by combining all the ingredients mentioned in the list and keep it readyheat a kadai with oil add mustard seeds cumin seeds and let it splutteradd onions and saute till they turn translucentadd tomatoes and cook till the tomatoes become soft and mushyadd ginger garlic paste red chilli powder turmeric powder coriander powder garam masala powder and salt to taste and mix well cook for 2 to 3 whistles and turn off the gas allow the pressure to release naturallyopen the lid of the kadhai and allow it to cook on low heat for 5 to 6 minutes until the tomato puree is cooked through and the raw smell goes awayonce done remove from the heat and transfer to a serving bowl and serve hotserve tomato puli ki sabzi along with steamed rice and phulka for a weekday lunch or dinner,https://www.archanaskitchen.com/tomato-puli-ki-sabzi-recipe-in-hindi
kashmiri style kachumber salad recipe,1 cup arhar dal split1 onion finely c

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Ragi Vermicelli Recipe starts with
Generated: Ragi Vermicelli Recipe starts with making the maharashtrian style kachumber salad recipe heat oil in a heavy bottomed pan add mustard seeds cumin seeds fennel seeds and let it splutteradd the chopped onions and saute till the onions turn translucent add the tomatoes and cook till they become soft and mushyadd in the ginger garlic paste red chilli powder turmeric powder coriander powder garam masala powder and salt mix well and allow it to cook for 5 to 6 minutes until the raw smell of the spices goes awayonce done turn off the heat and transfer to a serving bowl and garnish it with green chutney and serve hotserve the masoor dal along with steamed rice and phulka for a weekday lunch or dinner,https://www.archanaskitchen.com/masoor-dal-recipe-in-hindi
kashmiri style chakundar pulao recipe,4 potatoes aloo boiled and mashed2 green chillies finely chopped1 inch ginger grated2 cloves garlic minced1 teaspoon cayenne pepper powder1 bay lea

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Gongura Chicken Curry Recipe first prep all the ingredients
Generated: Gongura Chicken Curry Recipe first prep all the ingredients and keep them readyheat oil in a pan add mustard seeds cumin seeds and let it splutteradd onions ginger garlic paste and saute till the onions turn translucentadd tomato puree red chilli powder turmeric powder coriander powder garam masala powder and salt mix well and cook for 2 to 3 minutes until the tomatoes are soft and mushyadd the chicken pieces and mix everything well cook on low to medium heat for 3 to 4 whistles and turn off the heat allow the pressure to release naturally open the lid and allow it to rest for 10 minutes or till it is cooked throughonce done transfer the cooked chicken to a serving bowl and serve hotserve the spicy chicken curry recipe along with steamed rice and phulkas for a weekday lunch or dinner,https://www.archanaskitchen.com/spicy-chicken-curry-recipe-in-hindi
kashmiri style kachumber salad recipe,1 cup arhar dal spli

In [96]:
data.head()

,TranslatedRecipeName,TranslatedIngredients,PrepTimeInMins,CookTimeInMins,TotalTimeInMins,Servings,Cuisine,Course,Diet,TranslatedInstructions,URL
0,masala karela recipe,6 karela bitter gourd pavakkai deseededsalt to...,15,30,45,6,Indian,Side Dish,Diabetic Friendly,to begin making the masala karela recipedeseed...,https://www.archanaskitchen.com/masala-karela-...
1,spicy tomato rice recipe,21 2 cups rice cooked 3 tomatoes 3 teaspoons b...,5,10,15,3,South Indian Recipes,Main Course,Vegetarian,to make tomato puliogere first cut the tomatoe...,http://www.archanaskitchen.com/spicy-tomato-ri...
2,ragi semiya upma recipe ragi millet vermicelli...,112 cups rice vermicelli noodles thin1 onion s...,20,30,50,4,South Indian Recipes,South Indian Breakfast,High Protein Vegetarian,to begin making the ragi vermicelli recipe fir...,http://www.archanaskitchen.com/ragi-vermicelli...
3,gongura chicken curry recipe andhra style gong...,500 grams chicken2 onion chopped1 tomato chopp...,15,30,45,4,Andhra,Lunch,Non Vegeterian,to begin making gongura chicken curry recipe f...,http://www.archanaskitchen.com/gongura-chicken...
4,andhra style alam pachadi recipe adrak chutney...,1 tablespoon chana dal 1 tablespoon white urad...,10,20,30,4,Andhra,South Indian Breakfast,Vegetarian,to make andhra style alam pachadi first heat o...,https://www.archanaskitchen.com/andhra-style-a...


In [98]:
karela_text = data.loc[0,'TranslatedInstructions']
print(karela_text)

to begin making the masala karela recipedeseed the karela and slice do not remove the skin as the skin has all the nutrients add the karela to the pressure cooker with 3 tablespoon of water salt and turmeric powder and pressure cook for three whistles release the pressure immediately and open the lids keep asideheat oil in a heavy bottomed pan or a kadhai add cumin seeds and let it sizzleonce the cumin seeds have sizzled add onions and saute them till it turns golden brown in coloradd the karela red chilli powder amchur powder coriander powder and besan stir to combine the masalas into the kareladrizzle a little extra oil on the top and mix again cover the pan and simmer masala karela stirring occasionally until everything comes together well turn off the heattransfer masala karela into a serving bowl and serveserve masala karela along with panchmel dal and phulka for a weekday meal with your family


In [100]:
## Get the length of actual text
print(len(karela_text.split()))

158


In [102]:
### Checking Model performace with Gradio Interface

In [5]:
import gradio as gr
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [7]:
# Load the GPT-2 tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2-indian-food")  # Use the path where your model is saved

In [9]:
def generate_text(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=150, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [16]:
# Create a Gradio Interface
iface = gr.Interface(
    fn=generate_text,
    inputs="text",
    outputs="text",
    live=True,
    examples=[["To make a delicious curry, ", "you need to start with fresh vegetables and aromatic spices."]]
)

In [18]:
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
